# Introduction aux réseaux de neurones

## Partie 1 - Classification linéaire

In [ ]:
from __future__ import print_function

%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from dojo.dataset import Cifar10Dataset

### Chargement du dataset

Durant ce dojo, nous utiliserons le dataset CIFAR10. Ce dataset comprend 60000 images RGB 32 par 32 de 10 classes différentes:

0. airplane
1. automobile
2. bird
3. cat
4. deer
5. dog
6. frog
7. horse
8. ship
9. truck

Il existe également le dataset CIFAR100, qui a lui 100 classes distinctes. Pour plus de détails, voir le [site officiel](https://www.cs.toronto.edu/~kriz/cifar.html).

Évaluez les cellules suivante pour télécharger le jeu de données. Normalement, vous devriez voir qu'il y a 50000 images dans les données d'entraînement et 10000 images dans les données de test.

In [ ]:
cifar10 = Cifar10Dataset()

In [ ]:
print(cifar10.x_train.shape)
print(cifar10.y_train.shape)
print(cifar10.x_test.shape)
print(cifar10.y_test.shape)

In [ ]:
# Visualize some examples from the dataset.
# We show a few examples of training images from each class.
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(cifar10.y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(cifar10.x_train[idx].astype('uint8'))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

### Le problème de classification

Regardons la première image:

In [ ]:
print(cifar10.x_train[0])
print(cifar10.y_train[0])
plt.imshow(cifar10.x_train[0].astype('uint8'))

Cette image est comprise par un humain comme étant une grenouille. Par contre, pour un ordinateur, cette image n'est rien de plus q'un tableau de 32 x 32 x 3 nombres à virgule flottante. Le problème de classification d'image est donc le suivant:
 
 
> Étant donné des images $x_i$ et leurs étiquettes $y_i$, trouvez la fonction $f(X): x \rightarrow y$ qui maximise le bon nombre d'associations entre les images et les étiquettes et qui se généralise à des images inconnues.


La généralisation est un élément primordial en machine learning. C'est bien beau d'être capable de classifier correctement les données d'entraînement, mais il faut à tout pris éviter le phénomène d'*overfitting*. L'exemple classique est de trouver la courbe qui passe par une série de points. Si on a $n$ points, il est possible de trouver un polynôme de degré $n + 1$ qui passe parfaitement par tous les points. Par contre, cette courbe risque de moins bien performer qu'un modèle plus simple. pour passer à travers les autres points de la distribution que l'on n'as pas encore observé.

![overfitting](https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Overfitted_Data.png/300px-Overfitted_Data.png)


Généralement, en machine learning, on sépare les données en deux groupes: *training* et *testing*. Ainsi, on peut évaluer la performance du modèle sur les données inconnues en utilisant des données qui n'ont pas servi à l'entraînement.

### k-Nearest Neighbor

Un des algorithmes les plus simple pour résoudre le problème de classification est k-Nearest Neighbor (kNN). k est un hyper-paramètre de l'algorithme, c'est-à-dire un paramètre qui est choisi par le programmeur de l'algorithme en fonction de l'application requise. L'algorithme fonctionne comme suit:

Étant donné un point de test $x$, on regarde les $k$ voisins les plus proches et on procède à un votre de majorité sur la classe que devrait avoir $x$. Voici un exemple visuel où les données ont deux dimensions (x, y) et il y a trois classes possibles (bleu, rouge et vert).

![kNN](http://cs231n.github.io/assets/knn.jpeg)

Pour obtenir les voisins les plus proches, il faut d'abord se choisir une métrique de distance. Souvent, on utilise la distance L2 (ou distance euclidienne). Dans le cas 2D, on a:

$$
d = \sqrt{x^2 + y^2}
$$

On peut utiliser la même métrique pour des images en comparant pixel par pixel chacune des valeurs de r, g et b.

#### Exercice

Complétez le fichier k_nearest_neighbor.py situé dans le dossier dojo. Dans un premier temps, complétez la fonction `compute_distances_two_loops`. Vous pouvez viusaliser le résultat avec le code suivant:

> **NOTE** Nous utiliserons ici que le dixième du dataset. Les images rgb sont redimensionnés en vecteur.

In [ ]:
from dojo.k_nearest_neighbor import KNearestNeighbor

n_of_train = 5000
n_of_test = 500

x_train, y_train = cifar10.x_train[:n_of_train], cifar10.y_train[:n_of_train]
x_test, y_test = cifar10.x_test[:n_of_test], cifar10.y_test[:n_of_test]
x_train = x_train.reshape((n_of_train, -1))
x_test = x_test.reshape((n_of_test, -1))

print(x_train.shape)

classifier = KNearestNeighbor()
classifier.train(x_train, y_train)

dists = classifier.compute_distances_two_loops(x_test) # May take some time to compute
print(dists.shape)

In [ ]:
plt.imshow(dists, interpolation='none')
plt.show()

- Que représentent les colonnes claires? Les colonnes foncées?

Maintenant, implémentez la fonction `predict_labels`.

In [ ]:
# We use k = 1 (which is Nearest Neighbor).
y_test_pred = classifier.predict_labels(dists, k=3)

# Compute and print the fraction of correctly predicted examples
num_correct = np.sum(y_test_pred == y_test)
accuracy = float(num_correct) / n_of_test
print('Got %d / %d correct => accuracy: %f' % (num_correct, n_of_test, accuracy))

Votre algorithme devrait obtenir environ 27% d'accuracy. Vous pouvez jouer avec le paramètre $k$ pour regarder si les résultats s'améliorent ou non. Quelle est la meilleure valeur de k?

#### Discussion
- Peut-on utiliser kNN dans la forme présentée plus haut pour la classification de grands dataset?
- Nommer une limitation de l'algorithme.
- Comment avez-vous choisi la valeur de k? Avez-vous utilisé les données de test?


#### Classification linéaire

On peut décomposer la fonction $f$ en plusieurs fonctions. Au lieu d'avoir une fonction unique qui retourne directment l'étiquette $y$ de l'image, on peut avoir une fonction par classe du dataset et chaque fonction retourne le score associé à l'image en entrée qui indique à quel point il est probable que l'image est cette classe. Par exemple, si le dataset a seulement deux classes, 0 pour chien et 1 pour chat, nous aurions deux fonctions $f_0(X)$ et $f_1(X)$. Pour une image donnée $x_i$, si on a $f_0(x_i) = 0.6$ et $f_1(x_i) = 0.2$, on en déduit que l'image $x_i$  est un chien, car selon les fonctions, il est plus probable que l'image soit un chien. Voici une représentation visuelle des fonction $f_j(x)$. À noter, la figure est en 2D, mais dans le cas réel, les fonctions séparent les images dans un espace à $32 \times 32 \times 3 = 3072$ dimensions.

![classification figure](http://cs231n.github.io/assets/pixelspace.jpeg)

La classification linéaire est un modèle très simple pour représenter les fonctions $f_j(x)$. La formulation est la suivante:

$$
f_j(x) = w_j \cdot x + b_j
$$

> **NOTE** Pour simplifier les calculs, $x$ est redimensionné en un vecteur de 3072 éléments. Ainsi, il est très facile d'associer un point à chaque pixel et multiplier ces points à $x$, puisque cela se résume en un produit scalaire.

Dans la formule précédente, le vecteur $w_j$ représente les poids (ou *weights* en anglais) à associer à chaque pixel de l'image $x$ et $b_j$ est un scalaire qui représente le biais, c'est-à-dire si à priori il est plus probable d'observer la classe $j$.


Pour que le calcul soit plus efficace, il est possible de calculer tous les scores $f_j(x)$ en même temps. Il suffit de mettre tous les vecteurs $w_j$ dans une matrice $W$ où chaque ligne $j$ est $w_j$ et de faire le produit matriciel $Wx$, puis additionner un vecteur $b$ qui contient les biais. Cela revient exactement au même que de faire les produit scalaires séparemment. Voici une représentation visuelle cette opération:

![weight matrix](http://cs231n.github.io/assets/imagemap.jpg)


### Exercice

Trouvez les paramètres optimaux $w_j$ et $b_j$ qui permettent d'obtenir le meilleur score de classification.

> *INDICE* 

## Partie 2 - Backpropagation

Dans la partie précédente, nous avons vu une manière assez limitée de faire de la classification. En effet, 